In [15]:
from model import VAE
from trainer import Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

#dummy dataset
class DummyDataset(Dataset):
    def __init__(self, num_samples, input_shape):
        self.num_samples = num_samples
        self.input_shape = input_shape

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Generate a random x input
        x = torch.rand(self.input_shape)
        return x

# Parameters
num_samples = 100  # Number of samples in the dataset
input_shape = (1, 384, 320)  # Shape of the input x

# Create the dataset
dataset = DummyDataset(num_samples, input_shape)

# Create the dataloader
batch_size = 16
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)



In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vae = VAE(image_channels=1, latent_dim=4, device=device).to(device)

In [12]:


optimizer = optim.Adam(vae.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Generate a random tensor with the shape (b, 1, 384, 320)
batch_size = 8
random_tensor = torch.randn(batch_size, 1, 384, 320).to(device)

# Run a single epoch with the random tensor
vae_loss, rec_loss, kl_loss, g_loss = vae._run_epoch(random_tensor, optimizer, criterion)
print(f"VAE Loss: {vae_loss}, Reconstruction Loss: {rec_loss}, KL Loss: {kl_loss}, Generator Loss: {g_loss}")


VAE Loss: 2.0286290645599365, Reconstruction Loss: 1.192907452583313, KL Loss: 0.12935638427734375, Generator Loss: 0.706365168094635


In [13]:

class RandomDataset(Dataset):
    def __init__(self, size, shape):
        self.data = torch.randn(size, *shape)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

batch_size = 8
train_dataset = RandomDataset(100, (1, 384, 320))
val_dataset = RandomDataset(20, (1, 384, 320))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vae = VAE(image_channels=1, latent_dim=4, device=device).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
criterion = nn.MSELoss()

trainer = Trainer(vae, train_loader, val_loader, criterion, optimizer, device=device)
trainer.train(num_epochs=10)
